# Generer spørsmål om NKS Kunnskapsbasen

Denne notatboken inneholder en oppskrift på hvordan å generere _syntetiske_
spørsmål om NKS sin Kunnskapsbase.

Den beste måten å lage et datasettet på er ved å få ekspertbrukere til å
generere spørsmål og svar til den originale kunnskapsbasen og bruke dette
datasettet som en fasit å måle vektordatabasen opp i mot. Denne fremgangsmåten
er dessverre veldig tidkrevende og det kan ta veldig lang tid fra systemet er
satt opp til man får muligheten til å lage datasettet for å teste.

En annen måte å generere et tilsvarende syntetisk datasett, og det resten av
denne notatboken tar for seg, er å be en stor språkmodell om å generere
spørsmål på bakgrunn av den originale kunnskapsbasen. Denne fremgangsmåten er
relativt hurtig, den kan settes opp uten at man må koordinere med ekspertbrukere
og - utenom kostnaden ved å benytte språkmodellen, er noe alle Data Scientister i
NAV kan gjøre. Bakdelen med fremgangsmåten er at det kan være vanskelig å svare
på kvaliteten til spørsmålene. Tanken er med andre ord at et syntetisk datasett
som man kan måle vektordatabaser opp mot er bedre enn ingen datasett.

For å prøve å håndtere utfordringen med å vurdere kvaliteten på spørsmålene vil
vi i denne notatboken prøve å bruke `LangGraph`. Med `LangGraph` kan man sette
opp kjøretidsgrafer av språkmodeller som kan benyttes for å utføre forskjellige
oppgaver. For å generere syntetiske spørsmål så kan man jo tenke seg at vi først
vurderer om et avsnitt egner seg for å generere spørsmål. Når det er gjort så
kan man vurdere om spørsmålet i seg selv er godt eller om spørsmålet utelukkende
kan svares på ved hjelp av teksten som spørsmålet ble generert fra.

## NKS kunnskapsbase

Vi kommer i denne notebook-en til å benytte NKS sin kunnskapsbase som et
grunnlag. Det er gjort en veldig stor jobb allerede med å lese inn tekst fra
Salesforce og putte dette på BigQuery, som vi kommer til å benytte.

Måten teksten er organisert på er at hver rad i BigQuery inneholder en
kunnskapsartikkel som har flere faste seksjoner (kolonner i BigQuery). For å
enkelt kunne holde styr på hvilken seksjon som svarer til hva kommer vi til å be
språkmodellen om å lage spørsmål på bakgrunn av hver seksjon. Dette gir god
granularitet på spørsmålene og det blir enklere å sjekke at vektordatabasen(e)
klarer å hente ut riktig artikkel i ettertid.

In [ ]:
from rich.console import Console

# Opprett rich objekt for pen utskrift
console = Console()

## Håndtering av miljøvariabler

For å kunne koble til Azure OpenAI Services trenger vi et par miljøvariabler. Vi
benytter [`dotenv`](https://pypi.org/project/python-dotenv/) for dette slik at
det er konfigurerbart og repeterbart.

Pass på at du har en `.env` fil som inneholder:

- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv(verbose=True)
assert (
    "AZURE_OPENAI_ENDPOINT" in os.environ
), "Mangler Azure endpoint som miljøvariabel!"
assert (
    "AZURE_OPENAI_API_KEY" in os.environ
), "Mangler Azure API nøkkel som miljøvariabel!"

## Forberede database for generert datasett

For å enkelt kunne gjenbruke og slå sammen informasjon fra den originale
kunnskapsbasen og det genererte datasettet, setter vi opp en egen tabell på
BigQuery som kan inneholde spørsmål og tilhørende artikkel ID samt seksjonen
(kolonnen i den originale kunnskapsbasen) som kan svare på spørsmålet.

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="nks-aiautomatisering-prod-194a")
table_id = "nks-aiautomatisering-prod-194a.kunnskapsbase.syntetiske_sporsmal"

In [ ]:
from google.cloud.bigquery import SchemaField

# Database skjema på BigQuery for spørsmålsdatasettet
schema = [
    SchemaField("id", "STRING", mode="REQUIRED", description="ID til spørsmålet"),
    SchemaField(
        "knowledge_article_id",
        "STRING",
        mode="REQUIRED",
        description="ID til tilhørende kunnskapsartikkel",
    ),
    SchemaField(
        "knowledge_column",
        "STRING",
        mode="REQUIRED",
        description="Kolonne spørsmålet hører til",
    ),
    SchemaField(
        "created",
        "DATETIME",
        mode="REQUIRED",
        description="Tidspunkt spørsmålet ble lastet opp",
    ),
    SchemaField(
        "prompt",
        "STRING",
        mode="REQUIRED",
        description="Prompt som ble brukt for å generere spørsmål (uten selve teksten)",
    ),
    SchemaField(
        "question",
        "STRING",
        mode="REQUIRED",
        description="Spørsmål til kunnskapsartikkelen",
    ),
]

In [ ]:
# Opprett database hvis den ikke allerede finnes
table_ref = bigquery.TableReference.from_string(table_id=table_id)
table = client.create_table(table_ref, exists_ok=True)
if not table.schema:
    table.schema = schema
    client.update_table(table, fields=["schema"])
    console.print(f"[bold magenta]Opprettet/Endret database[/] '{table.full_table_id}'")
else:
    console.print("Spørsmålsdatabase [bold green]eksisterer allerede")

## Oppsett av kjøretidsgraf

Før vi kan prosessere kunnskapsartikler må vi først definere hvordan disse
artiklene skal prosesseres.

For å prosessere kunnskapsartiklene kommer vi til å opprette en kjøretidsgraf
som avgjør om innholdet i kunnskapsartikkelen egner seg for å generere et
spørsmål fra, deretter generere et spørsmål, før vi tilslutt prøver å vurdere om
spørsmålet sammen med teksten gir mening.

### Egner kunnskapsartikkel seg for spørsmålsgenerering?

Vi starter med å opprette en flyt for å vurdere om en kunnskapsartikkel egner
seg for å generere spørsmål.

In [ ]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field

BinaryScore = Literal["yes", "no"]
"""Svar fra språkmodell som enten er Ja eller Nei"""


# Svaret vi forventer fra språkmodellen når den evaluerer en kunnskapsartikkel
class EvaluateContent(BaseModel):
    """Evaluer innholdet i en kunnskapsartikkel for egnethet for spørsmålsgenerering."""

    score: BinaryScore = Field(
        description="Content is well suited for question generation, 'yes' or 'no'"
    )


# Modellen som skal evaluere kunnskapsartikler
llm_evaluator = AzureChatOpenAI(
    api_version="2023-03-15-preview", azure_deployment="gpt-4o-mini", temperature=0.0
).with_structured_output(EvaluateContent)

# Ledetekst for å evaluere en kunnskapsartikkel
prompt = """You are a grader assessing the relevance of a retrieved document for \
synthetic question generation. If the document contains enough text and meaningful \
questions could be generated based on the text, grade it as relevant. It does not need \
to be a stringent test. The goal is to try and filter out documents that are difficult \
to generate meaningful questions from. Give a binary score 'yes' or 'no' to indicate \
whether the document is relevant or not."""

eval_prompt = ChatPromptTemplate.from_messages(
    [("system", prompt), ("human", "Retrieved document:\n\n{document}")]
)
# LangChain runnable for å vurdere en kunnskapsartikkel
eval_grader = eval_prompt | llm_evaluator

In [ ]:
suitable = eval_grader.invoke(
    {
        "document": "NAVs samfunnsoppdrag er å bidra til sosial og økonomisk tryggleik og fremme overgang til arbeid og aktivitet. målet er å skape eit inkluderande samfunn, eit inkluderande arbeidsliv og ein velfungerande arbeidsmarknad."
    }
)
console.print(f"Er teksten godt nok for et spørsmål: {suitable}")

### Generere spørsmål

Det neste vi oppretter er en flyt for å generere spørsmål fra en
kunnskapsartikkel.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

llm_generator = AzureChatOpenAI(
    api_version="2023-03-15-preview",
    azure_deployment="gpt-4o-mini",
    temperature=0.5,
)
prompt = """You are a synthetic question generation system. You will be given a \
document and should generate a question that can be answered by the document alone. \
The question must be a complete sentence and must be in Norwegian. If given the \
question a user should be able to answer it based on the text in the given document."""

question_gen_prompt = ChatPromptTemplate.from_messages(
    [("system", prompt), ("human", "Document:\n\n{document}")]
)
# LangChain runnable for å generere et spørsmål
question_generator = question_gen_prompt | llm_generator | StrOutputParser()

In [ ]:
question = question_generator.invoke(
    {
        "document": "NAVs samfunnsoppdrag er å bidra til sosial og økonomisk tryggleik og fremme overgang til arbeid og aktivitet. Målet er å skape eit inkluderande samfunn, eit inkluderande arbeidsliv og ein velfungerande arbeidsmarknad."
    }
)
console.print(f"Forslag til spørsmål: [bold blue]{question}")

### Evaluer spørsmål fra kunnskapsartikkel

Før vi konkluderer med at et spørsmål er godt nok ber vi språkmodellen om å
vurdere dette faktumet.

In [ ]:
class GradeQuestion(BaseModel):
    """Evaluering av syntetisk spørsmål."""

    score: BinaryScore = Field(
        description="Question makes sense given the document, 'yes' or 'no'"
    )


llm_grader = AzureChatOpenAI(
    api_version="2023-03-15-preview", azure_deployment="gpt-4o-mini", temperature=0.0
).with_structured_output(GradeQuestion)
prompt = """You are a grader assessing whether a question makes sense \
given a document which must contain the answer to the question. Give a \
binary score 'yes' or 'no'. 'Yes' means that the question makes sense given \
the document."""
grade_prompt = ChatPromptTemplate.from_messages(
    [("system", prompt), ("human", "Document:\n\n{document}\n\nQuestion: {question}")]
)
question_grader = grade_prompt | llm_grader

In [ ]:
grade = question_grader.invoke(
    {
        "document": "NAVs samfunnsoppdrag er å bidra til sosial og økonomisk tryggleik og fremme overgang til arbeid og aktivitet. målet er å skape eit inkluderande samfunn, eit inkluderande arbeidsliv og ein velfungerande arbeidsmarknad.",
        "question": question,
    }
)
console.print(f"Er det et godt spørsmål: {grade}")

### Sett opp kjøretidsgrafen

For å koble dette sammen til et system oppretter vi en kjøretidsgraf slik at vi
kan inkorporere de foregående modellene.

In [ ]:
from typing import TypedDict

EvalResult = Literal["suitable", "not suitable"]


class GraphState(TypedDict):
    """Tilstanden til spørsmålsgenerering kjøretidsgraf."""

    document: str
    generated_question: str
    good_question: bool


# Definer flyten til kjøretidsgrafen
def evaluate_document(state: GraphState) -> EvalResult:
    """Evaluer en kunnskapsartikkel om den egner seg for å generere spørsmål."""
    # console.print("---[bold magenta]Evaluer dokument[/]---")
    doc = state["document"]
    suitable = eval_grader.invoke({"document": doc})
    if suitable.score == "yes":
        return "suitable"
    else:
        return "not suitable"


def generate_question(state: GraphState) -> GraphState:
    """Generer et spørsmål - Node i kjøretidsgrafen."""
    # console.print("---[bold magenta]Generer spørsmål[/]---")
    doc = state["document"]
    question = question_generator.invoke({"document": doc})
    return {"document": doc, "generated_question": question}


def grade_question(state: GraphState) -> GraphState:
    """Vurder om spørsmålet er godt sett i forhold til kunnskapsartikkelen."""
    # console.print("---[bold magenta]Vurder spørsmål[/]---")
    doc = state["document"]
    question = state["generated_question"]
    grade = question_grader.invoke({"question": question, "document": doc})
    return {
        "document": doc,
        "generated_question": question,
        "good_question": grade.score == "yes",
    }

In [ ]:
from langgraph.graph import END, START, StateGraph

# Kjøretidsgrafen
workflow = StateGraph(GraphState)

# Legg til noder i grafen
workflow.add_node("generate", generate_question)
workflow.add_node("grade", grade_question)
# Legg til kanter i grafen
workflow.add_conditional_edges(
    START,
    evaluate_document,
    {
        "suitable": "generate",
        "not suitable": END,
    },
)
workflow.add_edge("generate", "grade")
# Opprett flyt
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
inp = {
    "document": "NAVs samfunnsoppdrag er å bidra til sosial og økonomisk tryggleik og fremme overgang til arbeid og aktivitet. Målet er å skape eit inkluderande samfunn, eit inkluderande arbeidsliv og ein velfungerande arbeidsmarknad."
}
res = app.invoke(inp)
console.print(res)

## Last inn kunnskapsbasen

Vi benytter samme kode som `nks_vdb` for å laste inn kunnskapsbasen.

In [ ]:
from nks_kbs_analyse.knowledgebase import load

docs = list(load())

## Generer spørsmål

Tilslutt generer vi spørsmål fra kunnskapsbasen.

In [ ]:
import random

from langchain_core.documents import Document
from rich.progress import Progress

num_to_generate = 300
synthetic_questions: list[tuple[str, Document]] = []
with Progress() as progress:
    gen_task = progress.add_task("Lager spørsmål")
    while len(synthetic_questions) < num_to_generate:
        selected = random.choices(docs, k=min(num_to_generate, 10))
        qa_eval = app.batch([{"document": doc.page_content} for doc in selected])
        good_questions = [
            (qa["generated_question"], doc)
            for (qa, doc) in zip(qa_eval, selected)
            if "good_question" in qa and qa["good_question"]
        ]
        synthetic_questions.extend(good_questions)
        progress.update(
            gen_task, total=num_to_generate, completed=len(synthetic_questions)
        )
# Hvis vi har generert for mange spørsmål så kutter vi listen
synthetic_questions = synthetic_questions[:num_to_generate]

Klargjør data for opplasting til BigQuery.

In [ ]:
import datetime
import uuid

today = datetime.datetime.now(datetime.UTC)

rows = [
    dict(
        id=uuid.uuid4(),
        knowledge_article_id=doc.metadata["KnowledgeArticleId"],
        knowledge_column=doc.metadata["ContentColumn"],
        created=today,
        prompt=question_gen_prompt.messages[0].prompt.template,
        question=question,
    )
    for question, doc in synthetic_questions
]

Og last opp.

In [ ]:
from rich.prompt import Confirm

if Confirm.ask(f"Vil du laste opp {len(rows)} nye spørsmål til BigQuery?"):
    errors = client.insert_rows(rows=rows, table=table)
    if errors:
        console.print(
            f"[bold red]Det oppstod følgende feil ved opplasting:[/] {errors}"
        )
    else:
        console.print("[bold green]Opplasting ferdig!")
else:
    console.print("[bold yellow]Hoppet over opplasting")